In [ ]:
#Import libraries
from google.colab import drive 


#get drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Import libraries
import pandas as pd
import numpy as np
import datetime

from nltk.corpus import stopwords
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier

from sklearn import preprocessing, svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from textblob import Word

from io import StringIO
import os
import string
from scipy import spatial
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")
import calendar
calendar.setfirstweekday(6)
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()
from nltk.corpus import stopwords
stopword_list = stopwords.words('english')   

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#read csv to dataframe
cc_train = pd.read_csv('/content/drive/My Drive/Data Science/All Modules/5 Project1/Edureka_Consumer_Complaints_train.csv') 
cc_test = pd.read_csv('/content/drive/My Drive/Data Science/All Modules/5 Project1/Edureka_Consumer_Complaints_test.csv')

'''
Analyze the missing values in the columns
Before removing NAs from data, do check if there are columns which have too
many NaN.
'''
print(cc_train.isnull().sum().sort_values(ascending=False))
print(cc_test.isnull().sum().sort_values(ascending=False))

'''
Shows columns with missing values. Out of all the columns 'Consumer complaint narrative' column 
is the most important but cannot be manipulated as that will effect the accuracy of the results
'''

Tags                            308584
Consumer complaint narrative    302630
Company public response         290879
Consumer consent provided?      257230
Sub-issue                       219374
Sub-product                     103786
ZIP code                          2911
State                             2903
Product                              0
Issue                                0
Complaint ID                         0
Company                              0
Consumer disputed?                   0
Submitted via                        0
Date sent to company                 0
Company response to consumer         0
Timely response?                     0
Date received                        0
dtype: int64
Tags                            102629
Consumer complaint narrative    100692
Company public response          97146
Consumer consent provided?       85699
Sub-issue                        73250
Sub-product                      34683
ZIP code                           937
State       

"\nShows columns with missing values. Out of all the columns 'Consumer complaint narrative' column \nis the most important but cannot be manipulated as that will effect the accuracy of the results\n"

In [ ]:
#What is the number of unique values?
print(cc_train.nunique(axis=0))
#Shows unique values in each column

Date received                     1758
Product                             12
Sub-product                         47
Issue                               95
Sub-issue                           67
Consumer complaint narrative     55535
Company public response             10
Company                           3064
State                               62
ZIP code                         24181
Tags                                 3
Consumer consent provided?           4
Submitted via                        6
Date sent to company              1701
Company response to consumer         6
Timely response?                     2
Consumer disputed?                   2
Complaint ID                    358810
dtype: int64


In [ ]:
#What is the most frequently occurring categories in the Categorical Columns?
print('Date received : '+cc_train['Date received'].mode())
print('Product : '+cc_train['Product'].mode())
print('Sub-product : '+cc_train['Sub-product'].mode())
print('Issue : '+cc_train['Issue'].mode())
print('Sub-issue : '+cc_train['Sub-issue'].mode())
print('Consumer complaint narrative : '+cc_train['Consumer complaint narrative'].mode())
print('Company public response : '+cc_train['Company public response'].mode())
print('Company : '+cc_train['Company'].mode())
print('State : '+cc_train['State'].mode())
print('ZIP code : '+cc_train['ZIP code'].mode())
print('Tags : '+cc_train['Tags'].mode())
print('Consumer consent provided? : '+cc_train['Consumer consent provided?'].mode())
print('Submitted via : '+cc_train['Submitted via'].mode())
print('Date sent to company : '+cc_train['Date sent to company'].mode())
print('Company response to consumer : '+cc_train['Company response to consumer'].mode())
print('Timely response? : '+cc_train['Timely response?'].mode())
print('Consumer disputed? : '+cc_train['Consumer disputed?'].mode())
#Shows most frequently occuring values in each column

0    Date received : 2015-08-27
dtype: object
0    Product : Mortgage
dtype: object
0    Sub-product : Other mortgage
dtype: object
0    Issue : Loan modification,collection,foreclosure
dtype: object
0    Sub-issue : Account status
dtype: object
0    Consumer complaint narrative : This company co...
dtype: object
0    Company public response : Company chooses not ...
dtype: object
0    Company : Bank of America
dtype: object
0    State : CA
dtype: object
0    ZIP code : 300XX
dtype: object
0    Tags : Older American
dtype: object
0    Consumer consent provided? : Consent provided
dtype: object
0    Submitted via : Web
dtype: object
0    Date sent to company : 2015-11-13
dtype: object
0    Company response to consumer : Closed with exp...
dtype: object
0    Timely response? : Yes
dtype: object
0    Consumer disputed? : No
dtype: object


In [ ]:
#Can you identify the top issues raised by consumers in this dataset?
print(cc_train['Issue'].value_counts().head(10))
#Shows top 10 issues raised by consumers with count of number of customer complaints for each issue

Loan modification,collection,foreclosure    60185
Incorrect information on credit report      43825
Loan servicing, payments, escrow account    38456
Cont'd attempts collect debt not owed       27266
Account opening, closing, or management     17557
Disclosure verification of debt             12283
Communication tactics                       11457
Deposits and withdrawals                    11028
Application, originator, mortgage broker     8441
Billing disputes                             7244
Name: Issue, dtype: int64


In [ ]:
#Are there some products which receive a higher number of complaints?
print(cc_train.groupby(['Product'])['Complaint ID'].count().sort_values(ascending=False))
#All products with total number of complaints. Hence product with maximum complaints is 'Mortgage'

Product
Mortgage                   117066
Debt collection             65033
Credit reporting            60672
Credit card                 43114
Bank account or service     40667
Consumer Loan               13964
Student loan                11306
Money transfers              2502
Payday loan                  2425
Prepaid card                 1671
Other financial service       385
Virtual currency                5
Name: Complaint ID, dtype: int64


In [ ]:
#Do all the companies receive the same number of complaints?
print(cc_train.groupby(['Company'])['Complaint ID'].count().sort_values(ascending=False))
#shows number of complaints company-wise

Company
Bank of America                             34903
Wells Fargo & Company                       26490
JPMorgan Chase & Co.                        21677
Equifax                                     21489
Experian                                    20075
                                            ...  
Metro Capital Mortgage Corporation              1
Metro Denver Title, LLC                         1
Metropolitan Bureau of Collections, Inc.        1
Churchill Debt Settlement                       1
1st 2nd Mortgage Company Of NJ, Inc.            1
Name: Complaint ID, Length: 3064, dtype: int64


In [ ]:
#How are the complaints submitted - through which medium?
print(cc_train.groupby(['Submitted via'])['Complaint ID'].count())
#Shows number of complaints submitted via each medium

Submitted via
Email             188
Fax              5264
Phone           25802
Postal mail     23644
Referral        68450
Web            235462
Name: Complaint ID, dtype: int64


In [ ]:
#What is the geographical distribution of the complaints?
print(cc_train.groupby(['State'])['Complaint ID'].count().sort_values(ascending=False))
#shows number of complaints state-wise

State
CA    52368
FL    34567
TX    26683
NY    24554
GA    15886
      ...  
MH       19
MP       13
AS        9
PW        8
AA        7
Name: Complaint ID, Length: 62, dtype: int64


In [ ]:
#Feature Engineering - extract week number, name of the day from the given date columns. Was it the first week or last week of the month? 

def findDay(date):    
    year, month, day = (int(i) for i in date.split('-'))     
    dayNumber = calendar.weekday(year, month, day) 
    days =["Monday", "Tuesday", "Wednesday", "Thursday", 
                         "Friday", "Saturday", "Sunday"] 
    return (days[dayNumber])

def week_number_of_month(date_value):
    dat1=int(date_value.split('-')[0])
    dat2=int(date_value.split('-')[1])
    dat3=int(date_value.split('-')[2])
    #testdate=datetime.datetime(dat1,dat2,dat3)
    x = np.array(calendar.monthcalendar(dat1, dat2))
    return np.where(x==dat3)[0][0] + 1

date = cc_train['Date received'].values
day_r_train=[]
week_r_train=[]
for i in range(len(date)):
    day_r_train.append(findDay(date[i]))
    week_r_train.append(week_number_of_month(date[i]))
cc_train['Day received'] = day_r_train
cc_train['month received'] = cc_train['Date received'].apply(lambda x: x.split("-")[1])
cc_train['Week received'] = week_r_train

date = cc_train['Date sent to company'].values
day_s_train=[]
week_s_train=[]
for i in range(len(date)):
    day_s_train.append(findDay(date[i]))
    week_s_train.append(week_number_of_month(date[i]))
cc_train['Day sent to company'] = day_s_train
cc_train['month sent to company'] = cc_train['Date sent to company'].apply(lambda x: x.split("-")[1])
cc_train['Week sent to company'] = week_s_train

date = cc_test['Date received'].values
day_r_test=[]
week_r_test=[]
for i in range(len(date)):
    day_r_test.append(findDay(date[i]))
    week_r_test.append(week_number_of_month(date[i]))
cc_test['Day received'] = day_r_test
cc_test['month received'] = cc_test['Date received'].apply(lambda x: x.split("-")[1])
cc_test['Week received'] = week_r_test

date = cc_test['Date sent to company'].values
day_s_test=[]
week_s_test=[]
for i in range(len(date)):
    day_s_test.append(findDay(date[i]))
    week_s_test.append(week_number_of_month(date[i]))
cc_test['Day sent to company'] = day_s_test
cc_test['month sent to company'] = cc_test['Date sent to company'].apply(lambda x: x.split("-")[1])
cc_test['Week sent to company'] = week_s_test

#You can convert strings/object type columns to date_time data using pd.to_datetime
cc_train['Date received']= pd.to_datetime(cc_train['Date received'])
cc_train['Date sent to company']= pd.to_datetime(cc_train['Date sent to company'])
cc_test['Date received']= pd.to_datetime(cc_test['Date received'])
cc_test['Date sent to company']= pd.to_datetime(cc_test['Date sent to company'])

#What was the gap between the filing of complaints and the data being sent to the company?
cc_train['Difference between sent to and received'] = cc_train['Date sent to company'] - cc_train['Date received']
print(cc_train['Difference between sent to and received'])

print(cc_train)
print(cc_test)

0         0 days
1         0 days
2        14 days
3         1 days
4         0 days
           ...  
358805   -1 days
358806    4 days
358807    0 days
358808    4 days
358809   22 days
Name: Difference between sent to and received, Length: 358810, dtype: timedelta64[ns]
       Date received  ... Difference between sent to and received
0         2015-10-14  ...                                  0 days
1         2015-04-26  ...                                  0 days
2         2013-12-20  ...                                 14 days
3         2016-03-03  ...                                  1 days
4         2015-01-30  ...                                  0 days
...              ...  ...                                     ...
358805    2014-04-14  ...                                 -1 days
358806    2013-03-14  ...                                  4 days
358807    2013-02-01  ...                                  0 days
358808    2015-06-25  ...                                  4 days
3

In [ ]:
#Do the complaints rise in any specific month or day of the week?
print(cc_train.groupby(['month received'])['Complaint ID'].count().sort_values(ascending=False))
#Maximum complaints are recieved in the month of March
print(cc_train.groupby(['Day received'])['Complaint ID'].count().sort_values(ascending=False))
#Maximum complaints are recieved on Wednesday

month received
03    35578
04    34027
06    33635
05    33574
07    31431
01    30843
02    30672
08    28454
10    26257
12    25477
09    25393
11    23469
Name: Complaint ID, dtype: int64
Day received
Wednesday    69583
Tuesday      68750
Thursday     66618
Friday       59348
Monday       58817
Saturday     20752
Sunday       14942
Name: Complaint ID, dtype: int64


In [ ]:
#How do companies respond to the complaints?
print(cc_train.groupby(['Company public response'])['Complaint ID'].count().sort_values(ascending=False))
#Companies response on public portal is shown
#What are the most common responses received?
print(cc_train.groupby(['Company response to consumer'])['Complaint ID'].count().sort_values(ascending=False))
'''
Most common responses received on public portal is 'Company chooses not to provide a public response' and 
most common response to consumer is 'Closed with explanation'
'''

Company public response
Company chooses not to provide a public response                                                                           31526
Company has responded to the consumer and the CFPB and chooses not to provide a public response                            19268
Company believes it acted appropriately as authorized by contract or law                                                   12055
Company believes complaint caused principally by actions of third party outside the control or direction of the company     1091
Company believes the complaint is the result of a misunderstanding                                                          1075
Company disputes the facts presented in the complaint                                                                       1054
Company believes complaint is the result of an isolated error                                                                769
Company can't verify or dispute the facts in the complaint               

"\nMost common responses received on public portal is 'Company chooses not to provide a public response' and \nmost common response to consumer is 'Closed with explanation'\n"

In [ ]:
'''
Does responding to complaints in a timely manner alter the number of consumers that
disputed the company response and those that did not?
'''
print(cc_train[cc_train['Timely response?']=='Yes'].groupby(['Consumer disputed?'])['Timely response?'].count())
#Shows consumer disputes are less when response to complaints are sent in timely manner

Consumer disputed?
No     277732
Yes     74943
Name: Timely response?, dtype: int64


In [ ]:
'''
It does not make sense to use Consumer ID as a predictor
See whether you need to impute those values or need to drop that column altogether before you start removing NA observations from the entire data
It does not make sense to use ZIP CODES as a numeric variable
'''
ctrain=cc_train[['Date sent to company','Week sent to company','month sent to company','Day sent to company','Date received','Week received','month received','Day received','Product','Issue','Consumer complaint narrative','Company','Company response to consumer','Timely response?','Consumer disputed?']]
ctrain['Consumer complaint narrative'].fillna(ctrain['Issue'],inplace=True)
ctest=cc_test[['Date sent to company','Week sent to company','month sent to company','Day sent to company','Date received','Week received','month received','Day received','Product','Issue','Consumer complaint narrative','Company','Company response to consumer','Timely response?']]
ctest['Consumer complaint narrative'].fillna(ctest['Issue'],inplace=True)
ctest['Consumer disputed?']=''
print(ctrain.isnull().sum())
print(ctest.isnull().sum())

Date sent to company            0
Week sent to company            0
month sent to company           0
Day sent to company             0
Date received                   0
Week received                   0
month received                  0
Day received                    0
Product                         0
Issue                           0
Consumer complaint narrative    0
Company                         0
Company response to consumer    0
Timely response?                0
Consumer disputed?              0
dtype: int64
Date sent to company            0
Week sent to company            0
month sent to company           0
Day sent to company             0
Date received                   0
Week received                   0
month received                  0
Day received                    0
Product                         0
Issue                           0
Consumer complaint narrative    0
Company                         0
Company response to consumer    0
Timely response?                0
C

In [ ]:
#Converting Text data to Lowercase
ctrain['Consumer complaint narrative'] =ctrain['Consumer complaint narrative'].apply(lambda x: ' '.join([i.lower() for i in x.split()]))
ctest['Consumer complaint narrative'] =ctest['Consumer complaint narrative'].apply(lambda x: ' '.join([i.lower() for i in x.split()]))
#Removing Punctuations
ctrain['Consumer complaint narrative'] =ctrain['Consumer complaint narrative'].str.replace(r'[^\w\s]',"")
ctest['Consumer complaint narrative'] =ctest['Consumer complaint narrative'].str.replace(r'[^\w\s]',"")

In [ ]:
#Text standardization
'''
Below, we used three normalizazion dictionaries from these links :
http://people.eng.unimelb.edu.au/tbaldwin/etc/emnlp2012-lexnorm.tgz
http://luululu.com/tweet/typo-corpus-r1.txt
'''
dico = {}
dico2 = open('/content/drive/My Drive/Data Science/All Modules/5 Project1/doc2.txt', 'rb')
for word in dico2:
    word = word.decode('utf8')
    word = word.split()
    dico[word[0]] = word[1]
dico2.close()
dico3 = open('/content/drive/My Drive/Data Science/All Modules/5 Project1/doc3.txt', 'rb')
for word in dico3:
    word = word.decode('utf8')
    word = word.split()
    dico[word[0]] = word[1]
dico3.close()

In [ ]:
def txt_std(words):
    list_words = words.split()
    for i in range(len(list_words)):
        if list_words[i] in dico.keys():
            list_words[i] = dico[list_words[i]]
    return ' '.join(list_words)

In [ ]:
ctrain['Consumer complaint narrative'] =ctrain['Consumer complaint narrative'].apply(txt_std)
ctest['Consumer complaint narrative'] =ctest['Consumer complaint narrative'].apply(txt_std)

ctrain['Consumer complaint narrative'] =ctrain['Consumer complaint narrative'].str.replace(r"xx+\s","")
ctest['Consumer complaint narrative'] =ctest['Consumer complaint narrative'].str.replace(r"xx+\s","")

In [ ]:
#Removing Stopwords
stop = stopwords.words('english')
ctrain['Consumer complaint narrative'] =ctrain['Consumer complaint narrative'].apply(lambda x: ' '.join([i for i in x.split() if i not in stop]))
ctest['Consumer complaint narrative'] =ctest['Consumer complaint narrative'].apply(lambda x: ' '.join([i for i in x.split() if i not in stop]))

In [ ]:
#Correcting Spelling
'''
ctrain['Consumer complaint narrative'] =ctrain['Consumer complaint narrative'].apply(lambda x: str(TextBlob(x).correct()))
ctest['Consumer complaint narrative'] =ctest['Consumer complaint narrative'].apply(lambda x: str(TextBlob(x).correct()))
'''

"\nctrain['Consumer complaint narrative'] =ctrain['Consumer complaint narrative'].apply(lambda x: str(TextBlob(x).correct()))\nctest['Consumer complaint narrative'] =ctest['Consumer complaint narrative'].apply(lambda x: str(TextBlob(x).correct()))\n"

In [ ]:
#Lemmatizing
ctrain['Consumer complaint narrative'] =ctrain['Consumer complaint narrative'].apply(lambda x:' '.join([Word(i).lemmatize() for i in x.split()]))
print(ctrain['Consumer complaint narrative'])
ctest['Consumer complaint narrative'] =ctest['Consumer complaint narrative'].apply(lambda x:' '.join([Word(i).lemmatize() for i in x.split()]))


0                       incorrect information credit report
1         reply xxxxprivileged confidential informationd...
2                                                          
3                              disclosure verification debt
4                              disclosure verification debt
                                ...                        
358805                  incorrect information credit report
358806               loan modificationcollectionforeclosure
358807                                       payoff process
358808                      contd attempt collect debt owed
358809                  incorrect information credit report
Name: Consumer complaint narrative, Length: 358810, dtype: object


In [ ]:
X_train=ctrain[['Consumer complaint narrative']]
X_test=ctest[['Consumer complaint narrative']]
y_train=ctrain['Consumer disputed?']
y_test=ctest['Consumer disputed?']
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

358810
119606
358810
119606


In [ ]:
enc = preprocessing.LabelEncoder()
y_train = enc.fit_transform(y_train)
y_test = enc.fit_transform(y_test)

(358810,)


In [ ]:
#tf-idf verctor representation
tfidfvec = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000) 
tfidf_X_train = tfidf_vect.fit_transform((ctrain["Consumer complaint narrative"]))
tfidf_X_test = tfidf_vect.fit_transform((ctest["Consumer complaint narrative"]))
print(tfidf_X_train.shape) 
from sklearn.model_selection import GridSearchCV
clf = LogisticRegression()
lr_params = {'C':[int(x) for x in np.linspace(1,10,10)]}
grid_lr = GridSearchCV(estimator=clf,param_grid=lr_params,cv=5,n_jobs=-1)
grid_lr.fit(tfidf_X_train,y_train)

(358810, 5000)


GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
final_lr = LogisticRegression(C=5)
final_lr.fit(tfidf_X_train,y_train)

LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_predict = final_lr.predict(tfidf_X_train)
print(y_predict)
final_lr_predict = final_lr.predict(tfidf_X_test)
print(final_lr_predict)
DF = pd.DataFrame(final_lr_predict) 
DF=DF.replace(to_replace =0, value ="no")  
DF=DF.replace(to_replace =1, value ="yes")  
# save the dataframe as a csv file 
lr_accuracy = metrics.accuracy_score(final_lr_predict, y_test)
print ("Logistic Regression > Accuracy: ", lr_accuracy)

[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
Logistic Regression > Accuracy:  0.991171011487718


In [ ]:
print(len(y_test))
print(len(final_lr_predict))
print(DF.value_counts())

119606
119606
no     118550
yes      1056
dtype: int64


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, final_lr_predict,target_names=ctrain['Consumer disputed?'].unique()))
print(classification_report(y_train, y_predict,target_names=ctrain['Consumer disputed?'].unique()))

              precision    recall  f1-score   support

          No       1.00      0.99      1.00    119606
         Yes       0.00      0.00      0.00         0

    accuracy                           0.99    119606
   macro avg       0.50      0.50      0.50    119606
weighted avg       1.00      0.99      1.00    119606

              precision    recall  f1-score   support

          No       0.79      1.00      0.88    282638
         Yes       0.61      0.02      0.05     76172

    accuracy                           0.79    358810
   macro avg       0.70      0.51      0.46    358810
weighted avg       0.75      0.79      0.70    358810



In [ ]:
from sklearn.svm import LinearSVC
svc_model = LinearSVC()
svc_params = {'C':[0.01,0.1, 1, 10, 100, 1000]}
grid_svc = GridSearchCV(estimator=svc_model,param_grid=svc_params,cv=5,n_jobs=-1)
grid_svc.fit(tfidf_X_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=1000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=None, tol=0.0001, verbose=0),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10, 100, 1000]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
final_svc = LinearSVC(C=0.1)
final_svc.fit(tfidf_X_train,y_train)

LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
y_predict = final_svc.predict(tfidf_X_train)
print(y_predict)

final_svc_predict = final_svc.predict(tfidf_X_test)
svc_accuracy = metrics.accuracy_score(final_svc_predict, y_test)
print ("SVC > Accuracy: ", svc_accuracy)

[0 0 0 ... 0 0 0]
SVC > Accuracy:  0.9932612076317242


In [ ]:
print(classification_report(y_test, final_svc_predict,target_names=ctrain['Consumer disputed?'].unique()))
print(classification_report(y_train, y_predict,target_names=ctrain['Consumer disputed?'].unique()))

              precision    recall  f1-score   support

          No       1.00      0.99      1.00    119606
         Yes       0.00      0.00      0.00         0

    accuracy                           0.99    119606
   macro avg       0.50      0.50      0.50    119606
weighted avg       1.00      0.99      1.00    119606

              precision    recall  f1-score   support

          No       0.79      1.00      0.88    282638
         Yes       0.72      0.01      0.02     76172

    accuracy                           0.79    358810
   macro avg       0.76      0.51      0.45    358810
weighted avg       0.78      0.79      0.70    358810



In [ ]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, eta=0.3, silent=1, subsample=0.8)
xgb_model.fit(tfidf_X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, eta=0.3, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=50,
              min_child_weight=1, missing=None, n_estimators=80, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=4, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=1, subsample=0.8, verbosity=1)

In [ ]:
y_predict_boost = xgb_model.predict(tfidf_X_train)
print(y_predict_boost)

xgb_predict = xgb_model.predict(tfidf_X_test)
print(len(xgb_predict))
print(len(y_test))
xgb_accuracy = metrics.accuracy_score(xgb_predict, y_test)
print ("XGBoost > Accuracy: ", xgb_accuracy)

[0 1 0 ... 0 0 0]
119606
119606
XGBoost > Accuracy:  0.9956858351587713


In [ ]:
print(classification_report(y_test, xgb_predict,target_names=ctrain['Consumer disputed?'].unique()))
print(classification_report(y_train, y_predict_boost,target_names=ctrain['Consumer disputed?'].unique()))
DF_boost = pd.DataFrame(xgb_predict) 
DF_boost=DF_boost.replace(to_replace =0, value ="no")  
DF_boost=DF_boost.replace(to_replace =1, value ="yes")  
# save the dataframe as a csv file 
DF.to_csv("/content/drive/My Drive/Data Science/All Modules/5 Project1/prediction.csv",index=False)

              precision    recall  f1-score   support

          No       1.00      1.00      1.00    119606
         Yes       0.00      0.00      0.00         0

    accuracy                           1.00    119606
   macro avg       0.50      0.50      0.50    119606
weighted avg       1.00      1.00      1.00    119606

              precision    recall  f1-score   support

          No       0.82      1.00      0.90    282638
         Yes       1.00      0.17      0.29     76172

    accuracy                           0.82    358810
   macro avg       0.91      0.59      0.60    358810
weighted avg       0.86      0.82      0.77    358810

